Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# Using a Trained Model for Realtime Inference

If your system requires low-latency processing (to process a single document or small set of documents quickly), then the realtime inference is the right choice. In this notebook, we will demonstrate how to make a prediction of an input sentence with a trained [fastText](https://fasttext.cc/) model.

The outline of this notebook is as follows:

- Visualize the pipeline associated with the pipeline run.
- Get a step run with information from the visualization.
- Get the port with the trained model from the step run.
- Download the model from the port and register it to the workspace.
- Deploy the model to local/ACI/AKS.

## Prerequisites
If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the configuration Notebook located at https://github.com/Azure/MachineLearningNotebooks first. This sets you up with a working config file that has information on your workspace, subscription id, etc.


In [1]:
import os
from azureml.core import Model, Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.environment import Environment
from azureml.core.experiment import Experiment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, AciWebservice, AksWebservice
from azureml.pipeline.wrapper import PipelineRun

### Connect to workspace
Create a workspace object from the existing workspace. Workspace.from_config() reads the file config.json and loads the details into an object named workspace.


In [2]:
workspace = Workspace.from_config('config.json')
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id,
      workspace.compute_targets.keys(), sep='\n')

DesignerDRI_EASTUS
DesignerDRI
eastus
74eccef0-4b8d-4f83-b5f9-fa100d155b22
dict_keys(['aks-dev', 'aks-dev2', 'attached-aks', 'default', 'compute', 'cpu-cluster', 'aml-compute'])


### Get a list of experiment names from the workspace

In [3]:
exp_name_list = [exp.name for exp in Experiment.list(workspace)]
exp_name_list

['sample10',
 'sample5',
 'sample5-realtime',
 'simple10-batch',
 'pythonscript',
 'Data_dependency',
 'clement',
 'new_module',
 'test_module2',
 'test_m',
 'module_SDK_local_module_test',
 'fasttext_pipeline']

### Choose the experiment you want with its name

In [4]:
experiment_name = "fasttext_pipeline"
experiment = Experiment(workspace, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
fasttext_pipeline,DesignerDRI_EASTUS,Link to Azure Machine Learning studio,Link to Documentation


### Choose the latest run

In [5]:
# azureml.pipeline.core.run.PipelineRun
run = experiment.get_runs().__next__()
run

Experiment,Id,Type,Status,Details Page,Docs Page
fasttext_pipeline,2353d38d-c191-4045-b0a7-e943c4437ffa,azureml.PipelineRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


### Get a PipelineRun object

In [6]:
run_id = run.id
# azureml.pipeline.wrapper.PipelineRun
pipeline_run = PipelineRun(experiment, run_id)
pipeline_run

Experiment,Id,Type,Status,Details Page,Docs Page
fasttext_pipeline,2353d38d-c191-4045-b0a7-e943c4437ffa,azureml.PipelineRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


### Visualize the pipeline

In [7]:
# To do
# pipeline_run.visualize()

### Get a StepRun object

In [8]:
# step_run_id will be from the visualization result.
step_run_id = '28956fb7-79a2-496e-a09a-64cdacb1771e'
step_run = pipeline_run.get_step_run(step_run_id)
step_run

Experiment,Id,Type,Status,Details Page,Docs Page
fasttext_pipeline,28956fb7-79a2-496e-a09a-64cdacb1771e,azureml.StepRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


### Download the trained model from the port

In [9]:
# name will be from the visualization result.
# get_port() should supports three kinds of names: (1)the_better_model (2)The better model (3)The_better_model
port = step_run.get_port(name='The_better_model')
saved_path = port.download(overwrite=True)
print('model save at: {}'.format(saved_path))

Downloaded azureml/28956fb7-79a2-496e-a09a-64cdacb1771e/The_better_model/label.txt, 1 files out of an estimated total of 3
Downloaded azureml/28956fb7-79a2-496e-a09a-64cdacb1771e/The_better_model/word_to_index.json, 2 files out of an estimated total of 3
Downloaded azureml/28956fb7-79a2-496e-a09a-64cdacb1771e/The_better_model/BestModel, 3 files out of an estimated total of 3
model save at: /tmp/azureml/28956fb7-79a2-496e-a09a-64cdacb1771e/The_better_model


### Register the trained model for deployment

In [10]:
model_path = saved_path
model = Model.register(workspace, model_path=saved_path, model_name='model_for_deployment')
model

Registering model model_for_deployment


Model(workspace=Workspace.create(name='DesignerDRI_EASTUS', subscription_id='74eccef0-4b8d-4f83-b5f9-fa100d155b22', resource_group='DesignerDRI'), name=model_for_deployment, id=model_for_deployment:2, version=2, tags={}, properties={})

### Get an Environment object for Inference Configuration

In [11]:
env_list = Environment.list(workspace)
name = 'env_for_deployment'
# if the workspace doesn't contain the specific environment, then we'll register a new one.
if not name in env_list:
    # env_for_deployment.yaml describes dependencies your service need.
    file_path = 'deployment/env_for_deployment.yaml'
    env = Environment.from_conda_specification(name=name, file_path=file_path)
    env = env.register(workspace=workspace)
else:
    env = Environment.get(workspace=workspace, name=name)

### Get an InferenceConfig object for deployment
It represents configuration settings for a custom environment used for deployment


In [12]:
# entry_script defines the processing logic for the input
entry_script = 'fasttext_realtime_inference.py'
# source_directory is the path to the folder that contains all files to create the service image
source_directory = 'deployment'
inference_config = InferenceConfig(entry_script=entry_script, source_directory=source_directory,
                                   environment=env)

### Deploy to local

In [13]:
service_name = 'local-deploy-test'
models = [model]
port = 8892
deployment_config = LocalWebservice.deploy_configuration(port=port)
service_locally = Model.deploy(workspace=workspace, name=service_name, models=models, inference_config=inference_config,
                               deployment_config=deployment_config)
service_locally.wait_for_deployment(show_output=True)
print(service_locally.state)
print(service_locally.get_logs())

entry_script /mnt/batch/tasks/shared/LS_root/mounts/clusters/my-compute/code/Users/t-yangx/demo/azureml-designer-demo/deployment/scoring_for_deployment.py doesn't exist. entry_script should be path relative to  current working directory



Generating Docker build context.
Package creation Succeeded
Logging into Docker registry designerdriea2302d01.azurecr.io
Logging into Docker registry designerdriea2302d01.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM designerdriea2302d01.azurecr.io/azureml/azureml_ac0df2cc6dbad51226a8d176555b98dc
 ---> b2300022e9a5
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> c60d299dc972
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6Ijc0ZWNjZWYwLTRiOGQtNGY4My1iNWY5LWZhMTAwZDE1NWIyMiIsInJlc291cmNlR3JvdXBOYW1lIjoiZGVzaWduZXJkcmkiLCJhY2NvdW50TmFtZSI6ImRlc2lnbmVyZHJpX2Vhc3R1cyIsIndvcmtzcGFjZUlkIjoiZjZjNzVkZmYtMmMzMS00NDg2LTg0Y2ItMWRlZDMzODhlMzM3In0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in e3c1c9ae3a10
 ---> cac7d6c75452
Step 4/5 : RUN mv '/var/azureml-app/tmp80x_vcqu.py' /var/azureml-app/main.py
 ---> Running in 4c72d55eb709
 ---> 03717eede64d
Step 5/

### Deploy to ACI (Azure Container Instances)

In [14]:
service_name = 'aci-deploy-test'
models = [model]
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
service_aci = Model.deploy(workspace, service_name, models=models, inference_config=inference_config,
                           deployment_config=deployment_config, overwrite=True)
service_aci.wait_for_deployment(show_output=True)
print(service_aci.state)

Running........
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


### Deploy to AKS (Azure Kubernetes Service)

In [15]:
# choose an inference cluster
name = 'attached-aks'
compute_target_name_list = [target.name for target in AksCompute.list(workspace)]
if not name in compute_target_name_list:
    aks_resource_group = 'AmlStudioV2DRI'
    cluster_name = 'aks-dev-6node33512023a'
    attach_config = AksCompute.attach_configuration(resource_group=aks_resource_group, cluster_name=cluster_name)
    aks_target = ComputeTarget.attach(workspace, name, attach_config)
    aks_target.wait_for_completion(show_output=True)
aks_target = AksCompute(workspace, name)
aks_target

AksCompute(workspace=Workspace.create(name='DesignerDRI_EASTUS', subscription_id='74eccef0-4b8d-4f83-b5f9-fa100d155b22', resource_group='DesignerDRI'), name=attached-aks, id=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourceGroups/DesignerDRI/providers/Microsoft.MachineLearningServices/workspaces/DesignerDRI_EASTUS/computes/attached-aks, type=AKS, provisioning_state=Succeeded, location=eastus, tags=None)

In [18]:
# deploy to AKS (Azure Kubernetes Service)
service_name = 'aks-deploy-test'
models = [model]
# Only one type of Auth may be enabled
token_auth_enabled = True
auth_enabled = False if token_auth_enabled else True

deployment_config = AksWebservice.deploy_configuration(cpu_cores=1, memory_gb=1,
                                                       token_auth_enabled=token_auth_enabled,
                                                       auth_enabled=auth_enabled)
service_aks = Model.deploy(workspace, service_name, models, inference_config, deployment_config, aks_target, overwrite=True)
service_aks.wait_for_deployment(show_output=True)
print(service_aks.state)

Running.......
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy


### Consume the service

In [19]:
# consume
import requests
import json
from azureml.core.authentication import InteractiveLoginAuthentication

# Get a token to authenticate to the compute instance from remote
interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

# Create and submit a request using the auth header
headers = auth_header
# Add content type header
headers.update({'Content-Type': 'application/json'})

#### consume the service deployed to local

In [20]:
# your input
standard_sample_input = {'param': {'input_sentence': '当前的预测还不准确'}}
standard_sample_input = json.dumps(standard_sample_input)

# consume the service deployed to local
service = service_locally

response = requests.post(service.scoring_uri, data=standard_sample_input, headers=headers)
print(service.scoring_uri)
print(response)
print(response.content)
print(response.json())

http://localhost:8892/score
<Response [200]>
b'"game"'
game


#### consume the service deployed to ACI

In [21]:
# your input
standard_sample_input = {'param': {'input_sentence': '当前的预测还不准确'}}
standard_sample_input = json.dumps(standard_sample_input)

# consume the service deployed to ACI
service = service_aci

response = requests.post(service.scoring_uri, data=standard_sample_input, headers=headers)
print(service.scoring_uri)
print(response)
print(response.content)
print(response.json())

http://96c84315-acb4-4191-b4e8-ad3ccbd56315.eastus.azurecontainer.io/score
<Response [200]>
b'"game"'
game


#### consume the service deployed to AKS

In [22]:
# your input
standard_sample_input = {'param': {'input_sentence': '当前的预测还不准确'}}
standard_sample_input = json.dumps(standard_sample_input)

# consume the service deployed to AKS
service = service_aks
token, refresh_by = service.get_token()
headers['Authorization']=f'Bearer {token}'


response = requests.post(service.scoring_uri, data=standard_sample_input, headers=headers)
print(service.scoring_uri)
print(response)
print(response.content)
print(response.json())

http://52.170.37.14:80/api/v1/service/aks-deploy-test/score
<Response [200]>
b'"game"'
game
